In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from scipy import signal

# Helper functions used in other parts of the code
import HelperModules
import EngineeredFeatures

ds = xr.open_dataset('MyChallengePaleo/T2m_R1_ym_1stMill.nc')
T2m_R1 = ds.to_dataframe()

TSI = HelperModules.getForcingData('MyChallengePaleo/Solar_forcing_1st_mill.nc','TSI')
AOD = HelperModules.getForcingData('MyChallengePaleo/Volc_Forc_AOD_1st_mill.nc','AOD')

First let's pre-process the data using a scaling technique
---------
Namely, the "RobustScaler" from scikit-learn. Of course this does not add any information, but it may improve the performance of some subesquent tools.

In [ ]:
from sklearn.preprocessing import RobustScaler

In [ ]:
scaler = RobustScaler()
T2m_R1['T2m_transf'] = scaler.fit_transform(T2m_R1)
#scaler.inverse_transform(scaler.transform(T2m_R1))
T2m_R1

In [ ]:
fig = plt.figure(figsize=(15, 4))
ax1 = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax2 = fig.add_subplot(1, 2, 2, projection=ccrs.PlateCarree())
HelperModules.plotTimeSlice(T2m_R1['T2m'],1,fig,ax1)
HelperModules.plotTimeSlice(T2m_R1['T2m_transf'],1,fig,ax2);

Should we Cluster the data using k-Means, Agglomerative, or DBScan?
----------
Before going down the clustering path, it makes sense to ask: what dimensions make sense to consider?
 - Latitude and longitude for sure
 - Also time?
 - Does it make sense to engineer any features that encapsulate e.g. year-to-year changes?
 - Do we want to consider temperature, or temperature anomaly?
 - Do we want these clusters to be useful for some final goal?


Why I am skeptical about clustering
--------
What will clustering achieve in this case? Probably it will cluster times and latitudes that are the same temperature. This means it might cluster a particularly cold year in the tropics with a bunch of data from the temperate zones. What does this tell us? Probably nothing.a